In [2]:
import json
import requests
import psycopg2
from time import gmtime, strftime

### INSERT DATA INTO DATABASE (LOCALLY)

In [22]:
r = requests.get('http://wservice.viabicing.cat/v2/stations')
bicingJson = r.json()

In [26]:
#conexión a la base de datos
try:
    conn = psycopg2.connect("dbname='postgres' user='postgres' host='localhost' password='root'")
except:
    print("I am unable to connect to the database")

In [27]:
cursor = conn.cursor()
cursor.executemany("INSERT into bicing(id_station,lat,lon,slots,bikes) VALUES (%(id)s, %(latitude)s,%(longitude)s,%(slots)s,%(bikes)s)",bicingJson['stations'])
conn.commit()
cursor.close()

In [28]:
conn.close()

### GET DATA FROM DATABASE (EXTERNALLY)

In [5]:
try:
    conn = psycopg2.connect("dbname='postgres' user='postgres' host='192.168.1.205' password='root'")
except:
    print("I am unable to connect to the database")